# Instalar librerias

In [1]:
!pip install opencv-python
!pip install numpy
!pip install tensorflow
!pip install mediapipe
!pip install paho-mqtt

  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp310-cp310-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.14.1-cp310-cp310-win_amd64.whl (35 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl (1.5 MB)
  Using cached grpcio-1.60.0-cp310-cp310-win_amd64.whl.metadata (4.2 kB)
  Using cached tensorboard-2.15.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_estimator

# Importar librerias



In [2]:
import cv2
import numpy as np
import math, time
import tensorflow as tf
from tensorflow.keras.models import load_model
import mediapipe as mp
import paho.mqtt.client as paho
from paho import mqtt

In [3]:
# setting callbacks for different events to see if it works, print the message etc.
def on_connect(client, userdata, flags, rc, properties=None):
    print("CONNACK received with code %s." % rc)

# with this callback you can see if your publish was successful
def on_publish(client, userdata, mid, properties=None):
    print("client: " + str(client) + "userdata: " + str(userdata)+"mid: " + str(mid))

# print which topic was subscribed to
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

# print message, useful for checking if it was successful
def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))

In [4]:
# Modelo CCN
model = load_model('mp_mano_gestos')

# Clases
with open('gestos.names', 'r') as f:
    classNames = f.read().split('\n')

# Initializar Mediapipe
mp_dibujo = mp.solutions.drawing_utils
mp_manos = mp.solutions.hands

# Initializar la camara
cap = cv2.VideoCapture(0)

# Variables
enviar_comando = False
ret  = 'none'
sent = False

In [5]:
# MQTT
USER     = "seminario-IoT2024"
PASSWORD = "seminario-IoT2024"
ADDR     = "93fd2fab7309499aa37f1731f5d26a2a.s1.eu.hivemq.cloud"

# using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
# userdata is user defined data of any type, updated by user_data_set()
# client_id is the given name of the client
client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect

# enable TLS for secure connection
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
# set username and password
client.username_pw_set(USER, PASSWORD)
# connect to HiveMQ Cloud on port 8883 (default for MQTT)
client.connect(ADDR, 8883)

# setting callbacks, use separate functions like above for better visibility
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_publish = on_publish

# subscribe to all topics of encyclopedia by using the wildcard "#"
client.subscribe("Arduino/MQTT", qos=1)

(0, 1)

In [6]:
client.loop_start()
# Iniciando bucle de captura de video
with mp_manos.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Volteando horizontalmente para visualización natural
        image = cv2.flip(image, 1)

        # Configurando la bandera de escritura
        image.flags.writeable = False

        # Detecciones de mano
        results = hands.process(image)

        # Configurando la bandera de escritura nuevamente
        image.flags.writeable = True

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Contando los dedos
        finger_count = 0
        gesture = ''  # Inicializando la variable de gesto

        # Renderizando los resultados
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                # Mediciones de distancias
                thumb_tip = hand_landmarks.landmark[mp_manos.HandLandmark.THUMB_TIP]
                thumb_ip = hand_landmarks.landmark[mp_manos.HandLandmark.THUMB_IP]
                pinky_mcp = hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_MCP]

                thumb_tip_to_pinky_mcp = math.dist((thumb_tip.x, thumb_tip.y), (pinky_mcp.x, pinky_mcp.y))
                thumb_ip_to_pinky_mcp = math.dist((thumb_ip.x, thumb_ip.y), (pinky_mcp.x, pinky_mcp.y))

                # Pulgar
                if thumb_tip_to_pinky_mcp > thumb_ip_to_pinky_mcp:
                    finger_count += 1

                # Dedo índice
                if hand_landmarks.landmark[mp_manos.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.INDEX_FINGER_PIP].y:
                    finger_count += 1

                # Dedo medio
                if hand_landmarks.landmark[mp_manos.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.MIDDLE_FINGER_PIP].y:
                    finger_count += 1

                # Dedo anular
                if hand_landmarks.landmark[mp_manos.HandLandmark.RING_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.RING_FINGER_PIP].y:
                    finger_count += 1

                # Dedo meñique
                if hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_PIP].y:
                    finger_count += 1

                mp_dibujo.draw_landmarks(image, hand_landmarks, mp_manos.HAND_CONNECTIONS,
                                        mp_dibujo.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_dibujo.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )

                # Predicción de gestos
                landmarks = np.array([[int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])] for lm in hand_landmarks.landmark])
                landmarks = np.expand_dims(landmarks, axis=0)
                prediction = model.predict(landmarks)
                classID = np.argmax(prediction)
                gesture = classNames[classID]

        if gesture == 'thumbs down' and enviar_comando == True:
            enviar_comando = False
            command = 'detener'
            client.publish('Arduino/MQTT', command)  # Publish MQTT command
            sent = False

        if gesture == 'thumbs up' and enviar_comando == False:
            enviar_comando = True
            command = 'start'
            client.publish('Arduino/MQTT', command)  # Publish MQTT command

        if enviar_comando == True and finger_count == 5 and sent == False:
            # Enviar un comando para encender luces
            sent = True
            command = 'cinco'
            client.publish('Arduino/MQTT', command)  # Publish MQTT command
            time.sleep(0.01)

        # Mostrar el conteo de dedos y la predicción de gesto
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, f'Conteo de Dedos: {finger_count}', (10, 30), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Gesto: {gesture}', (10, 70), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Enviar Habilitado: {enviar_comando} && Enviado {sent}', (10, 100), font, 1, (0, 255, 0), 2, cv2.LINE_AA)


        # Mostrar el fotograma de salida final
        cv2.imshow('Seguimiento de Mano y Reconocimiento de Gesto', image)

        # Romper el bucle si se presiona la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Liberar la cámara web y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()
client.loop_stop()

CONNACK received with code Success.
Subscribed: 1 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x0000020867935C60>]
1/1 [==============================] - 0s 16ms/step


In [7]:
client.disconnect()

0